In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/allstate-claims-severity/sample_submission.csv
/kaggle/input/allstate-claims-severity/train.csv.zip
/kaggle/input/allstate-claims-severity/sample_submission.csv.zip
/kaggle/input/allstate-claims-severity/train.csv
/kaggle/input/allstate-claims-severity/test.csv
/kaggle/input/allstate-claims-severity/test.csv.zip


In [3]:
from datetime import datetime
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.feature_selection import RFECV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

In [4]:
def timer(start_time = None) :
    if not start_time :
        start_time = datetime.now()
        return start_time
    elif start_time :
        tmin, tsec = divmod((datetime.now() - start_time).total_seconds(), 60)
        print('Time taken in minutes and seconds :', tmin, round(tsec, 2))

In [6]:
train = pd.read_csv('/kaggle/input/allstate-claims-severity/train.csv', dtype = {'id' : np.str, 'loss' : np.float32})
y = np.array(train['loss'])
test = pd.read_csv('/kaggle/input/allstate-claims-severity/test.csv', dtype = {'id': np.str})

In [7]:
train.head()

,id,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14,loss
0,1,A,B,A,B,A,A,A,A,B,...,0.718367,0.335060,0.30260,0.67135,0.83510,0.569745,0.594646,0.822493,0.714843,2213.179932
1,2,A,B,A,A,A,A,A,A,B,...,0.438917,0.436585,0.60087,0.35127,0.43919,0.338312,0.366307,0.611431,0.304496,1283.599976
2,5,A,B,A,A,B,A,A,A,B,...,0.289648,0.315545,0.27320,0.26076,0.32446,0.381398,0.373424,0.195709,0.774425,3005.090088
3,10,B,B,A,B,A,A,A,A,B,...,0.440945,0.391128,0.31796,0.32128,0.44467,0.327915,0.321570,0.605077,0.602642,939.849976
4,11,A,B,A,B,A,A,A,A,B,...,0.178193,0.247408,0.24564,0.22089,0.21230,0.204687,0.202213,0.246011,0.432606,2763.850098


In [8]:
trainc = train.drop(['id', 'loss'], axis = 1)
testc = test.drop(['id'], axis = 1)
train_shape = trainc.shape[0]
test_shape = testc.shape[0]
train_test = pd.concat((trainc,testc)).reset_index(drop = True)
all_features = [x for x in trainc.columns]
cat_features = [x for x in trainc.select_dtypes(include = ['object']).columns]
num_features = [x for x in trainc.select_dtypes(exclude = ['object']).columns]
print('categorical features len :', len(cat_features))
print('numerical features len :', len(num_features))

categorical features len : 116
numerical features len : 14


In [10]:
for c in range(len(cat_features)) :
    train_test[cat_features[c]] = train_test[cat_features[c]].astype('category').cat.codes
    
trainc = train_test.iloc[:train_shape, :]
testc = train_test.iloc[train_shape:, :]
X = np.array(trainc)
Xt = np.array(testc)

In [ ]:
# to test the features properly, it i good idea to change n_estimators = 200, step = 2, and max_Depth = 20 (or remove it).
# It will take longer time, around 5 -10 hrs.

rf = RandomForestRegressor(n_estimators = 100, max_features = 'sqrt', max_depth = 12, n_jobs = -1)
rfecv = RFECV(estimator = rf, step = 10, cv = KFold(y.shape[0], n_splits = 5, shuffle = False, random_state = 2020),
              scoring = 'neg_mean_absolute_error', verbose = 2)

start_time = timer(None)
rfecv.fit(X, y)
timer(start_time)

/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_split.py:297: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


Fitting estimator with 130 features.
Fitting estimator with 120 features.
Fitting estimator with 110 features.
Fitting estimator with 100 features.
Fitting estimator with 90 features.
Fitting estimator with 80 features.
Fitting estimator with 70 features.
Fitting estimator with 60 features.
Fitting estimator with 50 features.
Fitting estimator with 40 features.
Fitting estimator with 30 features.
Fitting estimator with 20 features.
Fitting estimator with 10 features.
Fitting estimator with 130 features.
Fitting estimator with 120 features.
Fitting estimator with 110 features.
Fitting estimator with 100 features.
Fitting estimator with 90 features.
Fitting estimator with 80 features.
Fitting estimator with 70 features.
Fitting estimator with 60 features.
Fitting estimator with 50 features.
Fitting estimator with 40 features.
Fitting estimator with 30 features.
Fitting estimator with 20 features.
Fitting estimator with 10 features.
Fitting estimator with 130 features.
Fitting estimator w

Fitting estimator with 80 features.
Fitting estimator with 70 features.
Fitting estimator with 60 features.
Fitting estimator with 50 features.
Fitting estimator with 40 features.
Fitting estimator with 30 features.
Fitting estimator with 20 features.
Fitting estimator with 10 features.
Fitting estimator with 130 features.
Fitting estimator with 120 features.
Fitting estimator with 110 features.
Fitting estimator with 100 features.
Fitting estimator with 90 features.
Fitting estimator with 80 features.
Fitting estimator with 70 features.
Fitting estimator with 60 features.
Fitting estimator with 50 features.
Fitting estimator with 40 features.
Fitting estimator with 30 features.
Fitting estimator with 20 features.
Fitting estimator with 10 features.
Fitting estimator with 130 features.
Fitting estimator with 120 features.
Fitting estimator with 110 features.
Fitting estimator with 100 features.
Fitting estimator with 90 features.
Fitting estimator with 80 features.
Fitting estimator wi

Fitting estimator with 30 features.
Fitting estimator with 20 features.
Fitting estimator with 10 features.
Fitting estimator with 130 features.
Fitting estimator with 120 features.
Fitting estimator with 110 features.
Fitting estimator with 100 features.
Fitting estimator with 90 features.
Fitting estimator with 80 features.
Fitting estimator with 70 features.
Fitting estimator with 60 features.
Fitting estimator with 50 features.
Fitting estimator with 40 features.
Fitting estimator with 30 features.
Fitting estimator with 20 features.
Fitting estimator with 10 features.
Fitting estimator with 130 features.
Fitting estimator with 120 features.
Fitting estimator with 110 features.
Fitting estimator with 100 features.
Fitting estimator with 90 features.
Fitting estimator with 80 features.
Fitting estimator with 70 features.
Fitting estimator with 60 features.
Fitting estimator with 50 features.
Fitting estimator with 40 features.
Fitting estimator with 30 features.
Fitting estimator wi

Fitting estimator with 110 features.
Fitting estimator with 100 features.
Fitting estimator with 90 features.
Fitting estimator with 80 features.
Fitting estimator with 70 features.
Fitting estimator with 60 features.
Fitting estimator with 50 features.
Fitting estimator with 40 features.
Fitting estimator with 30 features.
Fitting estimator with 20 features.
Fitting estimator with 10 features.
Fitting estimator with 130 features.
Fitting estimator with 120 features.
Fitting estimator with 110 features.
Fitting estimator with 100 features.
Fitting estimator with 90 features.
Fitting estimator with 80 features.
Fitting estimator with 70 features.
Fitting estimator with 60 features.
Fitting estimator with 50 features.
Fitting estimator with 40 features.
Fitting estimator with 30 features.
Fitting estimator with 20 features.
Fitting estimator with 10 features.
Fitting estimator with 130 features.
Fitting estimator with 120 features.
Fitting estimator with 110 features.
Fitting estimator w

Fitting estimator with 60 features.
Fitting estimator with 50 features.
Fitting estimator with 40 features.
Fitting estimator with 30 features.
Fitting estimator with 20 features.
Fitting estimator with 10 features.
Fitting estimator with 130 features.
Fitting estimator with 120 features.
Fitting estimator with 110 features.
Fitting estimator with 100 features.
Fitting estimator with 90 features.
Fitting estimator with 80 features.
Fitting estimator with 70 features.
Fitting estimator with 60 features.
Fitting estimator with 50 features.
Fitting estimator with 40 features.
Fitting estimator with 30 features.
Fitting estimator with 20 features.
Fitting estimator with 10 features.
Fitting estimator with 130 features.
Fitting estimator with 120 features.
Fitting estimator with 110 features.
Fitting estimator with 100 features.
Fitting estimator with 90 features.
Fitting estimator with 80 features.
Fitting estimator with 70 features.
Fitting estimator with 60 features.
Fitting estimator wi

In [ ]:
print('Optimal number of features :', rfecv.n_features_)
sel_features = [f for f, s in zip(all_features, rfecv.support_) if s]
print('Selected features are :', sel_features)

In [ ]:
# plot number of features vs cv scores

plt.figure()
plt.xlabel('number of features tested x 10')
plt.ylabel('cross val score - neg_mean_abs_error')
plt.plot(range(1, rfecv.grid_scores_) + 1, rfecv_grid_scores_)
plt.savefig('Allstate-RFECV.png')
plt.show()

In [ ]:
ranking = pd.DataFrame({'features' : all_features})
ranking['Rank'] = np.asarray(rfecv.ranking_)
ranking.sort_values('Rank', inplace = True)
ranking.to_csv('./Allstate-RFECV-ranking.csv', index=False)
print(' Ranked features saved:  Allstate-RFECV-ranking.csv')

In [ ]:
score = round(-np.max(rfecv.grid_scores_), 3)
test['loss'] = rfecv.predict(Xt)
test = test[['id', 'loss']]
now = datetime.now()
sub_file = 'submission_5xRFECV-RandomForest_' + str(score) + '_' + str(now.strftime("%Y-%m-%d-%H-%M")) + '.csv'
print("\n Writing submission file: %s" % sub_file)
test.to_csv(sub_file, index=False)

In [ ]:
#  